In [1]:
import os

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

from sklearn.pipeline import make_pipeline

### Data Prerequisites

In [2]:
for data_ in ['2021-01', '2021-02']:
    data_path = 'data/green_tripdata_'+data_+'.parquet'
    if not os.path.exists(data_path):
        url = f"https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_{data_}.parquet"
        try:
            os.system(f"wget {url} -P \data")
        except:
            print(f"{url} not available")

### Set MLflow local tracking system

In [ ]:
# import mlflow

# mlflow.set_tracking_uri("sqlite:///mlflow.db")
# mlflow.set_experiment("green-taxi-duration")

# `mlflow ui --backend-store-uri sqlite:///mlflow.db --default-artifact-root file:"<mlruns_file_location" -h 0.0.0.0 -p 8000`

### Set MLflow in remote server - scenario 3 of Week2 notes

In [3]:
import mlflow
import os

# os.environ("AWS_PROFILE") = ""

TRACKING_SERVER_HOST = "ec2-44-201-142-183.compute-1.amazonaws.com"
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://ec2-44-201-142-183.compute-1.amazonaws.com:5000'


In [4]:
mlflow.list_experiments()

[<Experiment: artifact_location='s3://ride-predictions-mlflow-artifacts/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='s3://ride-predictions-mlflow-artifacts/1', experiment_id='1', lifecycle_stage='active', name='ride-duration-models', tags={}>]

### Train the model

In [5]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [6]:
df_train = read_dataframe('data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('data/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [7]:
mlflow.set_experiment('ride-duration-models')
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.7558229919200725


### Get the model

In [8]:
mlflow.list_experiments()

[<Experiment: artifact_location='s3://ride-predictions-mlflow-artifacts/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='s3://ride-predictions-mlflow-artifacts/1', experiment_id='1', lifecycle_stage='active', name='ride-duration-models', tags={}>]

In [9]:
import mlflow

### Open MLflow:
URI = '0b929405a77e4097ab00f5db2a117412'
model_id = f'runs:/{URI}/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(model_id)
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.sklearn
  run_id: 0b929405a77e4097ab00f5db2a117412

In [12]:
URI = "s3://ride-predictions-mlflow-artifacts/1/0b929405a77e4097ab00f5db2a117412/artifacts/model"
loaded_model = mlflow.pyfunc.load_model(URI)
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.sklearn
  run_id: 0b929405a77e4097ab00f5db2a117412